In [ ]:
!pip install saliency

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 2.0 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim
import torch
from PIL import Image
import matplotlib.pyplot as plt

import saliency.core as saliency

In [ ]:

def compare_folders_ssim(folder1_path, folder2_path):
    # List files in both folders
    folder1_files = os.listdir(folder1_path)
    folder2_files = os.listdir(folder2_path)

    # Initialize a list to store SSIM scores
    ssim_scores = []

    # Loop through the files and calculate SSIM scores
    for file1, file2 in zip(folder1_files, folder2_files):
        # Load arrays from files
        array1 = torch.load(os.path.join(folder1_path, file1))
        array2 = torch.load(os.path.join(folder2_path, file2))

        # Denormalize the array to [0, 255]
        array1 = (array1 * 255).astype(np.uint8)
        array2 = (array2 * 255).astype(np.uint8)

        # Create an image from the array
        image1 = Image.fromarray(array1)
        image2 = Image.fromarray(array2)

        # Convert images to grayscale
        image_gray1 = image1.convert('L')
        image_gray2 = image2.convert('L')

        # Convert grayscale images to arrays
        gray_array1 = np.array(image_gray1)
        gray_array2 = np.array(image_gray2)

        # Calculate SSIM
        ssim_score, _ = ssim(gray_array1, gray_array2, full=True)

        # Store SSIM score
        ssim_scores.append(ssim_score)

    # Compute the mean of SSIM scores
    mean_ssim = np.mean(ssim_scores)

    return mean_ssim


In [ ]:
def compare_folders_ssim_pair(folder1_path, folder2_path):
    # List files in both folders
    folder1_files = os.listdir(folder1_path)
    folder2_files = os.listdir(folder2_path)

    # Initialize a list to store SSIM scores
    ssim_scores = []

    # Loop through the files and calculate SSIM scores
    for file1, file2 in zip(folder1_files, folder2_files):

        #NON XRAI

        # Load arrays from files
        #array1 = torch.load(os.path.join(folder1_path, file1))
        #array2 = torch.load(os.path.join(folder2_path, file2))

        #gray_image1 = saliency.VisualizeImageGrayscale(array1)
        #gray_image2 = saliency.VisualizeImageGrayscale(array2)

        #XRAI
        array1 = torch.load(os.path.join(folder1_path, file1))
        array2 = torch.load(os.path.join(folder2_path, file2))

        gray_image1 = array1
        gray_image2 = array2

        ssim_score, _ = ssim(gray_image1, gray_image2, full=True)

        # Store SSIM score
        ssim_scores.append(ssim_score)

    # Compute the mean of SSIM scores
    mean_ssim = np.mean(ssim_scores)

    return mean_ssim


In [ ]:
# Parent folder containing different folders for Pneumonia
pneumonia_parent_path = '/content/drive/MyDrive/Saliency Map Research 2023/Sensitivity/Saliency Maps/Pneumonia'

target_folders = ['logits', '7c', '7b', '7a', '6e', '6d', '6c', '6b', '6a', '5d', '5c', '5b']
techniques = ['XRAI']
 #['Grad', 'Smooth Grad', 'IG', 'Smooth IG', 'Guided Backprop', 'GradCAM', 'Guided GradCAM', 'ScoreCAM', 'GradCAM++']

# Outer loop to iterate through each technique
for technique in techniques:
    orig_folder_path = os.path.join(pneumonia_parent_path, 'orig', technique)

    # Initialize a list to store SSIM scores
    ssim_scores_list = []

    # Check if the technique requires different comparison function
    if technique in ['Grad', 'Smooth Grad', 'IG', 'Smooth IG', 'XRAI', 'Guided Backprop', 'Guided GradCAM']:
        compare_function = compare_folders_ssim_pair
    else:
        compare_function = compare_folders_ssim

    # Compare 'orig' folder arrays with each target folder in 'Pneumonia'
    for folder_name in target_folders:
        current_folder_path = os.path.join(pneumonia_parent_path, folder_name, technique)
        mean_ssim = compare_function(orig_folder_path, current_folder_path)
        print(f"Mean SSIM Score between orig/{technique} and {folder_name}: {mean_ssim}")
        ssim_scores_list.append(mean_ssim)

    # Create a new directory to save the results
    results_directory = '/content/drive/MyDrive/Saliency Map Research 2023/Sensitivity/Results'

    # Save the SSIM scores list to a text file
    results_file_path = os.path.join(results_directory, f'{technique}_ssim_scores.txt')
    with open(results_file_path, 'w') as f:
        for score in ssim_scores_list:
            f.write(f"{score}\n")

    print(f"SSIM scores for {technique} saved to:", results_file_path)

Mean SSIM Score between orig/XRAI and logits: 0.7760993217345343
Mean SSIM Score between orig/XRAI and 7c: 0.5366105338995202
Mean SSIM Score between orig/XRAI and 7b: 0.4258456288053759
Mean SSIM Score between orig/XRAI and 7a: 0.4262236611390757
Mean SSIM Score between orig/XRAI and 6e: 0.39041841665416244
Mean SSIM Score between orig/XRAI and 6d: 0.3761806092753963
Mean SSIM Score between orig/XRAI and 6c: 0.372634862820802
Mean SSIM Score between orig/XRAI and 6b: 0.36991168507454586
Mean SSIM Score between orig/XRAI and 6a: 0.36953543156223295
Mean SSIM Score between orig/XRAI and 5d: 0.36851874137343954
Mean SSIM Score between orig/XRAI and 5c: 0.36814783224754655
Mean SSIM Score between orig/XRAI and 5b: 0.3680518869801714
SSIM scores for XRAI saved to: /content/drive/MyDrive/Saliency Map Research 2023/Sensitivity/Results/XRAI_ssim_scores.txt


In [ ]:
def z_score_normalization_0_to_1(data):
    # Calculate the mean and standard deviation of the array
    mean = np.mean(data)
    std = np.std(data)

    # Calculate the Z-scores for each element in the array
    z_scores = (data - mean) / std

    # Apply linear transformation to map Z-scores between 0 and 1
    min_z = np.min(z_scores)
    max_z = np.max(z_scores)
    normalized_data = (z_scores - min_z) / (max_z - min_z)

    return normalized_data


In [ ]:
array1 = torch.load('/content/drive/MyDrive/Saliency Map Research 2023/Sensitivity/Saliency Maps/Pneumonia/orig/Grad/01cd2ba5-2baf-44b2-bf15-ee57e1ea4599.dcm.jpg.png_saliency_map.pt')
array2 = torch.load('/content/drive/MyDrive/Saliency Map Research 2023/Sensitivity/Saliency Maps/Pneumonia/5c/Grad/01cd2ba5-2baf-44b2-bf15-ee57e1ea4599.dcm.jpg.png_saliency_map.pt')

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Denormalize the array to [0, 255]
denormalized_array = (array1 * 255).astype(np.uint8)

# Create an image from the array
image = Image.fromarray(denormalized_array)

# Display the image using matplotlib
plt.imshow(image)
plt.axis('off')  # Turn off axes ticks and labels
plt.show()

In [ ]:
# Denormalize the array to [0, 255]
denormalized_array = (array2 * 255).astype(np.uint8)

# Create an image from the array
image2 = Image.fromarray(denormalized_array)

# Display the image using matplotlib
plt.imshow(image2)
plt.axis('off')  # Turn off axes ticks and labels
plt.show()